# JPEG Compression with Zonal Coding Quantization
stough 202-
DIP 8.9

In this activity we're going to synthesize a bit of the [JPEG](https://en.wikipedia.org/wiki/JPEG) encoding standard, specifically block transform encoding and zonal coding quantization. We will look to understand how the quantization affects the image reconstruction.

Read up in DIP 8.9 or [elsewhere](https://en.wikipedia.org/wiki/JPEG#JPEG_codec_example) if needed.

## Imports
We're going to take advantage of both [`view_as_blocks`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.view_as_blocks) and the [Discrete Cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform) for this.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair,
                       vis_surface)

from wavelet_utils import (make_haar_matrix,
                           make_random_basis,
                           make_klt_basis,
                           make_dct_matrix,
                           make_standard_matrix,
                           vis_blocks)

from skimage.util import view_as_blocks
from skimage.util import montage
from skimage.transform import resize, rescale

## JPEG Compression
Basically
- Take the image in $[0,255]$ form and subtract 128. This is to center around 0. Make sure you change the `dtype` when you do this, or else you won't be able to represent negatives.
- Transform each 8x8 block using the DCT transform matrix (see `make_dct_matrix`)
- Quantize the transform coefficients according the [Q matrix](https://en.wikipedia.org/wiki/JPEG#Quantization). This is, divide the block coefficients and store the result in integer form.
- These quantized transform blocks represent how the compressed image would be stored or transmitted. We won't go further than just having this representation, as the point of the exercise is to understand how this quantization affects the reconstruction. 
- Reconstruction: for each transform block, remultiply according to the Q matrix and then invert the transform process, to reconstruct the as-though-compressed image block.

We'll define the Q matrix for ease of use.

In [2]:
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]])

In [3]:
I = plt.imread('../dip_pics/cat_small.png')
I = I[...,:3]    # alpha
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
I = 255*I
I = I - 128
arr_info(I)

((512, 512, 3), dtype('float32'), -128.0, 124.0)

In [5]:
block_shape = (8,8,3)

view = view_as_blocks(I, block_shape)
arr_info(view)

((64, 64, 1, 8, 8, 3), dtype('float32'), -128.0, 124.0)

In [6]:
block_view = np.reshape(view, [view.shape[0]*view.shape[1]] + list(block_shape))
arr_info(block_view)

((4096, 8, 8, 3), dtype('float32'), -128.0, 124.0)

In [7]:
plt.figure()
plt.imshow(block_view[64, ..., 0], cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Forward Transformation

In [8]:
H = make_dct_matrix(8)
vis_blocks(H)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
BT = np.zeros_like(block_view)

for i, block in enumerate(block_view):
    CT = np.zeros_like(block)
    
    for chan in range(3):
        CT[..., chan] = np.matmul(H, np.matmul(block[..., chan], H.T))
        
    BT[i] = CT

In [10]:
plt.figure()
plt.imshow(BT[64, ..., 0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Quantization Stage

In [11]:
QTs = np.zeros_like(BT)

for i, CT in enumerate(BT):
    temp = np.zeros_like(CT)
    for chan in range(3):
        temp[..., chan] = np.int64(CT[..., chan]/Q)
        temp[..., chan] = np.float32(Q*temp[..., chan])
    QTs[i] = temp

In [12]:
arr_info(QTs)

((4096, 8, 8, 3), dtype('float32'), -1008.0, 976.0)

In [13]:
plt.figure()
plt.imshow(QTs[64, ..., 0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reconstruct

In [14]:
block_view_rec = np.zeros_like(block_view)

for i, CT in enumerate(QTs):
    BR = np.zeros_like(CT)
    
    for chan in range(3):
        BR[..., chan] = np.matmul(H.T, np.matmul(CT[..., chan], H))
        
    block_view_rec[i] = BR

In [15]:
Ir = montage(block_view_rec, grid_shape=view.shape[:2], multichannel=True)
arr_info(Ir)

((512, 512, 3), dtype('float32'), -142.96017, 127.22406)

In [16]:
plt.figure()
Ir_final = np.clip(Ir+128, 0, 255)/255
plt.imshow(Ir_final)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
vis_pair((I+128)/255, Ir_final)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Exercise: Interactive Quantization Demo
Put together an interactive demo that shows the original image, its current reconstruction, and a view of the currently quantized transform coefficients. This demo should then interact with a `FloatSlider` that affects the multiple applied to the Q quantization matrix. This demo would show how you could increase the compression (increasing the multiple) and what that increased compression would do to the image reconstruction.

In [18]:
from ipywidgets import VBox, HBox, FloatSlider

In [19]:
I = plt.imread('../dip_pics/firewatcher.png')
I = I[...,:3]    # alpha
I = 255*I
I = I - 128    # center at 0

In [20]:
block_shape = (8,8,3)
view = view_as_blocks(I, block_shape)
block_view = np.reshape(view, [view.shape[0]*view.shape[1]] + list(block_shape))
H = make_dct_matrix(8)

In [21]:
def quantize(BT, factor):
    QTs = np.zeros_like(BT)
    for i, CT in enumerate(BT):
        temp = np.zeros_like(CT)
        for chan in range(3):
            temp[..., chan] = np.int64(CT[..., chan]/(Q*factor))
            temp[..., chan] = np.float32(Q*temp[..., chan]*factor)
        QTs[i] = temp
    return QTs

In [22]:
BT = np.zeros_like(block_view)
for i, block in enumerate(block_view):
    CT = np.zeros_like(block)
    for chan in range(3):
        CT[..., chan] = np.matmul(H, np.matmul(block[..., chan], H.T))
    BT[i] = CT

QTs = quantize(BT, 1)
block_view_rec = np.zeros_like(block_view)
for i, CT in enumerate(QTs):
    BR = np.zeros_like(CT)
    for chan in range(3):
        BR[..., chan] = np.matmul(H.T, np.matmul(CT[..., chan], H))
    block_view_rec[i] = BR
Ir = montage(block_view_rec, grid_shape=view.shape[:2], multichannel=True)
plt.ioff()
Ir_final = np.clip(Ir+128, 0, 255)/255

In [23]:
factor = 1
coe_slider = FloatSlider(
    orientation='horizontal',
    value=factor,
    min=0.5,
    max=15.0,
    step=0.1,
    description='Coefficient'
)

In [24]:
fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args)

adisp = ax[0].imshow((I+128)/255)
rdisp = ax[1].imshow(Ir_final)

ax[0].set_title('Original')
rtext = ax[1].set_title(f'coefficient: {coe_slider.value:.02f}')

def reconstruct_image(change):
    global Ir_final, BT, QTs, CT, adisp, rdisp, rtext
    QTs = quantize(BT, coe_slider.value)
    block_view_rec = np.zeros_like(block_view)
    for i, CT in enumerate(QTs):
        BR = np.zeros_like(CT)
        for chan in range(3):
            BR[..., chan] = np.matmul(H.T, np.matmul(CT[..., chan], H))
        block_view_rec[i] = BR
    Ir = montage(block_view_rec, grid_shape=view.shape[:2], multichannel=True)
    Ir_final = np.clip(Ir+128, 0, 255)/255
    
    rdisp.set_array(Ir_final)
    rtext.set_text(f'coefficient: {coe_slider.value:.02f}')
    fig.canvas.draw()
    fig.canvas.flush_events()

coe_slider.observe(reconstruct_image, names='value')
VBox([HBox([coe_slider]), fig.canvas])